# Network Analysis Lab

## Group: 21

### Members: Max Kretschmer and Kara Adamcik


When we are interested in the interactions among entities of interest and the structures that emerge from their relations, we can model the entities and their relations as a graph, which is sufficiently defined by its nodes and edges. Considering a network as a graph allows us to study the overall connectivity of the contained nodes, identify formation of clusters, or position particular nodes in their neighborhood. In this tutorial, we will get started with network analysis. But please note: this is just a start. Network analysis is becoming its own network science.

## 🛒 1. Prepare 

First, we will need to assemble our tools: Apart from Altair, we will use [NetworkX](https://networkx.github.io), a powerful network-analysis library. As a bridge between Altair and NetworkX, we are using [nx_altair](https://github.com/Zsailer/nx_altair). 

You will have to install `nx_altair` and maybe also `networkx` via `!pip install`, which the notebook actually executes as a shell command (note the preceding exclamation mark). So let's get this out of the way first, after which we import all the libraries we will be using in this notebook:

In [1]:
!pip install networkx nx_altair

import altair as alt
import networkx as nx
import nx_altair as nxa
import pandas as pd


### Generate a random graph

To get started quickly, we can create a random graph. The `fast_gnp_random_graph` method uses the Erdős-Rényi model to generate a graph according to two main parameters: the number of nodes **`n`** and the probability that a given pair of nodes is connected **`p`**:

In [2]:
G = nx.fast_gnp_random_graph(n=36, p=.2)

# show graph with nx_altair
nxa.draw_networkx(G)

alt.LayerChart(...)

💡 *Play with the parameters of `fast_gnp_random_graph`, but go easy on the `n` *

### Create a network from scratch

You can also create a graph by manually adding nodes and edges with the respective methods `add_edge` and `add_node`:

In [3]:
G = nx.Graph()

G.add_node("Ada")
G.add_node("Bob")
G.add_node("Cai")
G.add_node("Don")
G.add_node("Eva")

G.add_edge("Ada", "Bob")
G.add_edge("Ada", "Cai")
G.add_edge("Ada", "Eva")
G.add_edge("Bob", "Cai")
G.add_edge("Bob", "Don")
G.add_edge("Cai", "Don")

nxa.draw_networkx(G)

alt.LayerChart(...)

Note that you actually do not need to add a node, if it is part of an edge. 

💡 *Comment out or remove the `add_node()` statements above (lines 3-7) and check the result!*

An even more compact way of creating a graph, adding nodes, and edges is by simply passing a list of edge tuples when creating the graph:

In [4]:
G = nx.Graph([("Ada", "Bob"),
              ("Ada", "Cai"),
              ("Ada", "Eva"),
              ("Bob", "Cai"),
              ("Bob", "Don"),
              ("Cai", "Don")])

### Add attributes to nodes and edges

You can attach attributes to nodes and edges, either when adding them to the graph or later:


In [5]:
G = nx.Graph()

G.add_node(1, time='3pm')
G.nodes[1]

{'time': '3pm'}

In [6]:
G.nodes[1]['room'] = 5842
G.nodes.data()

NodeDataView({1: {'time': '3pm', 'room': 5842}})

Here the nodes are defined as numbers, but as we have seen above NetworkX can also take strings as ids

In addition, you can add attributes to edges. A common way to distinguish between different strengths of connections is to assign weights to edges:

In [7]:
G.add_edge(1, 2, weight=4.7)

You can also add or edit edge attributes later:

In [8]:
G.edges[1, 2]['weight'] = 3.2
G.edges.data()

EdgeDataView([(1, 2, {'weight': 3.2})])

### Load a network dataset 

Network data can come in many formats, and thankfully NetworkX can read and write many of them, including [GEXF](https://networkx.github.io/documentation/stable/reference/readwrite/gexf.html), [GML](https://networkx.github.io/documentation/stable/reference/readwrite/gml.html), [GraphML](https://networkx.github.io/documentation/stable/reference/readwrite/graphml.html) and [JSON](https://networkx.github.io/documentation/stable/reference/readwrite/json_graph.html) (as used by D3.js). 

The co-occurrence network of characters in the novel *Les Misérables* (1862) by Victor Hugo serves as a common example dataset for network visualization. Let's load and import it. NetworkX does not (yet) load and parse JSON transparently (as Pandas does so elegantly). Therefore, we need to include the package `json` to get this done:


In [9]:
import json

# with open('https://gist.githubusercontent.com/TieJean/bfad558a88b11246884a56d86738b52b/raw/57f21d740bf61012e414b910f349907be00a9667/miserables.json') as f:
#     les_mis = json.load(f)
import requests
request = requests.get('https://gist.githubusercontent.com/TieJean/bfad558a88b11246884a56d86738b52b/raw/57f21d740bf61012e414b910f349907be00a9667/miserables.json')
les_mis = request.json()

# we specify that the dataset is not a multigraph, there are no self-loops
# or multiedges, multiple edges between nodes
G = nx.readwrite.json_graph.node_link_graph(les_mis, multigraph=False)

nxa.draw_networkx(G, node_tooltip="name")

alt.LayerChart(...)

## 🕸 2. Process

Once we have a graph representation of a network, we can carry out a range of processing steps, for example, to count its elements and generate some graph-theoretical metrics.


### Counting nodes and edges

For a start, we can get the number of nodes and edges:

In [10]:
G.number_of_nodes()

77

In [11]:
G.number_of_edges()

254

There is a convenient **`info()`** function (akin to the Pandas function of the same name) it gives us some basic stats, including the average degree, i.e., the number of connections an average node in this graph has:

In [12]:
print(nx.info(G))

Graph with 77 nodes and 254 edges


### Graph metrics

Networks may vary a lot by their number of edges in relationship to the number of nodes, which is considered the **`density()`** of a network. The density of a network ranges between 0 and 1: from no connections whatsover to all every node is connected to every other node. It thus also relates to the probability of two random nodes being connected, which we have used further above!

In [13]:
density = nx.density(G)
print("Network density:", density)

Network density: 0.08680792891319207


Another metric that interests network scientists is the shortest path between a given pair of nodes, i.e., we might want to know the shortest connection between two characters in the les mis network:

In [14]:
names = ("Napoleon", "Jondrette")

ids = [x for x,y in G.nodes(data=True) if y['name'] in names]
ids

[1, 46]

In [15]:
path = nx.shortest_path(G, source=ids[0], target=ids[1])

print("Shortest path between {} & {}:".format(names[0], names[1]), path)

Shortest path between Napoleon & Jondrette: [1, 0, 11, 48, 47, 46]


💡 *Of course, these are just their nondescript ids. What would it take to know their names?*

In [16]:
names = [ G.nodes[id]["name"] for id in path ]
names

['Napoleon', 'Myriel', 'Valjean', 'Gavroche', 'Mme.Burgon', 'Jondrette']

The length of above path equals the edges between these nodes, which equals the number of elements in the list minus 1:

In [17]:
print("Length of above path:", len(path)-1)

Length of above path: 5


### Measures of centrality

A considerable part of network analysis is devoted to identifying the most important actors in a network according to their position in the network, which is also referred to as centrality. One very simple centrality measure for relative importance of a node is its **degree**, i.e., the number of connections it has to other nodes. NetworkX can calculate this measure for all nodes of the graph in a snap:

In [18]:
degrees = dict(G.degree(G.nodes()))

# save the degrees as a node attribute
nx.set_node_attributes(G, degrees, 'degree')

# check what has been saved
G.nodes.data()

NodeDataView({0: {'name': 'Myriel', 'group': 1, 'degree': 10}, 1: {'name': 'Napoleon', 'group': 1, 'degree': 1}, 2: {'name': 'Mlle.Baptistine', 'group': 1, 'degree': 3}, 3: {'name': 'Mme.Magloire', 'group': 1, 'degree': 3}, 4: {'name': 'CountessdeLo', 'group': 1, 'degree': 1}, 5: {'name': 'Geborand', 'group': 1, 'degree': 1}, 6: {'name': 'Champtercier', 'group': 1, 'degree': 1}, 7: {'name': 'Cravatte', 'group': 1, 'degree': 1}, 8: {'name': 'Count', 'group': 1, 'degree': 1}, 9: {'name': 'OldMan', 'group': 1, 'degree': 1}, 10: {'name': 'Labarre', 'group': 2, 'degree': 1}, 11: {'name': 'Valjean', 'group': 2, 'degree': 36}, 12: {'name': 'Marguerite', 'group': 3, 'degree': 2}, 13: {'name': 'Mme.deR', 'group': 2, 'degree': 1}, 14: {'name': 'Isabeau', 'group': 2, 'degree': 1}, 15: {'name': 'Gervais', 'group': 2, 'degree': 1}, 16: {'name': 'Tholomyes', 'group': 3, 'degree': 9}, 17: {'name': 'Listolier', 'group': 3, 'degree': 7}, 18: {'name': 'Fameuil', 'group': 3, 'degree': 7}, 19: {'name': 'B

We save the node degrees with the respective nodes in the graph, so we can revisit it, when we visualize the graph. But let us do a little ‘connectivity contest’. *Who are the characters with the most encounters with other characters?*

In [19]:
# reverse sort of the degrees
sorted_degree = sorted(degrees.items(), key=lambda x: x[1], reverse=True)
# above x[1] refers to degree as items() returns both keys and values in tuples

print("Top 10 nodes by degree:\n")

for d in sorted_degree[:10]:
  print( " - {} has {} neighbors".format(G.nodes[d[0]]["name"], d[1]) )

Top 10 nodes by degree:

 - Valjean has 36 neighbors
 - Gavroche has 22 neighbors
 - Marius has 19 neighbors
 - Javert has 17 neighbors
 - Thenardier has 16 neighbors
 - Fantine has 15 neighbors
 - Enjolras has 15 neighbors
 - Courfeyrac has 13 neighbors
 - Bossuet has 13 neighbors
 - Bahorel has 12 neighbors


Node degree only takes into account the immediate neighbors. Another metric for relative importance of a node in a network is **betweenness centrality**, which takes into account all the shortest paths (see above) between all possible node pairs that go pass the respective node. This already indicates that this centrality measure is much more related to the overall importance of each node to the entire network, not just the immediate neighbors.

In [20]:
between = nx.betweenness_centrality(G)

nx.set_node_attributes(G, between, 'between')

sorted_between = sorted(between.items(), key=lambda x: x[1], reverse=True)

print("Top 10 nodes by between:\n")

for d in sorted_between[:10]:
  print( " - {}: {} ".format(G.nodes[d[0]]["name"], d[1]) )

Top 10 nodes by between:

 - Valjean: 0.5699890527836184 
 - Myriel: 0.17684210526315788 
 - Gavroche: 0.16511250242584766 
 - Marius: 0.132032488621946 
 - Fantine: 0.12964454098819422 
 - Thenardier: 0.07490122123424225 
 - Javert: 0.05433155966478436 
 - Mlle.Gillenormand: 0.047598927875243675 
 - Enjolras: 0.0425533568221771 
 - Tholomyes: 0.04062934817733579 


Betweenness centrality also ranges between 0 and 1, where 0 means that no shortest paths and 1 all shortest paths go through this node. Notice how the selection and order of characters actually differ considerably!

## 🥗 3. Present

After adding several centrality measures, we now have a fertile ground to generate insightful network visualizations that go beyond the default encoding you have already seen above. For the following steps we will continue with the Les Misérables network. 

### Force-directed layouts

Let's generate a network visualization and add a few quick customizations that might help to make sense of the network. First let's give the chart a bit more breathing room via the `properties()` call and add `tooltips` to the nodes:

In [21]:
nxa.draw_networkx(G, node_tooltip='name').properties(width=500, height=500)

alt.LayerChart(...)

The `spring_layout` is the default layout; it is an implementation of the Fruchterman-Reingold algorithm and takes several parameters. You can adjust it by generating the `pos` by hand.

💡 *Have a look into the [documentation](https://networkx.github.io/documentation/stable/reference/generated/networkx.drawing.layout.spring_layout.html?highlight=spring_layout#networkx.drawing.layout.spring_layout)*

In [22]:
pos = nx.spring_layout(G, iterations=100)

nxa.draw_networkx(G, pos, node_tooltip='name').properties(width=500, height=500)

alt.LayerChart(...)

### Custom graph layouts

Since nx_altair generates the network visualization as Altair charts, we can actually decide much more about the visual encoding. The first choice is how the visual variable x/y-position is used. In other words, how should the layout of the network be generated. The `spring_layout` is the default graph layout that nx_altair uses, but NetworkX provides several other [graph layouts](https://networkx.github.io/documentation/stable/reference/drawing.html#module-networkx.drawing.layout).


### Customizing visual encodings

In addition, to the node positions the other visual variables can also be changed to reflect design choices and the network data. Let's start with `node_size` and `node_color`:

In [23]:
nxa.draw_networkx(G, pos=pos,
    node_size='degree:Q',
    node_color='group:N',
    cmap = "category10", # pass colormap that is used
    node_tooltip='name:N',
    linewidths=0, # remove borders from circles
).properties(width=500, height=500)

alt.LayerChart(...)

### Multi-view visualization

Last but not least, we can harness the interactive powers of Altair to combine the force-directed layout with other visualizations that operate on node or edge data. In the following, we combine a regular force-directed layout with a barchart representing node groups. Selections in one view have impact on the respective other.

In [24]:
# rectangular selection in the network view
selection = alt.selection_interval(encodings=['x', 'y'])

# group selection in the bar chart
selection2 = alt.selection(type="multi", fields=['group'])


# first we create the force-directed layout
chart = nxa.draw_networkx( G, pos=pos,
    node_size=100,
    node_color='group:N',
    width='value:Q',
    node_tooltip='name',
    linewidths=0
)

# get node and edge layers from chart
edges = chart.layer[0]
nodes = chart.layer[1]

# group numbers (needed to keep bar chart stable during selections)
groups = list(range(1,10))

# separate color definition used across both charts
color = alt.Color('group:N', scale=alt.Scale(domain=groups), legend=None)

# adjust node opacity and fill color according to selections
nodes = nodes.encode(
    opacity=alt.condition(selection, alt.value(1), alt.value(0.25)),
    fill=alt.condition(selection2, color, alt.value('lightgray')),
).add_selection(selection,selection2)

# interactive bar chart 
bars = alt.Chart(nodes.data).mark_bar().encode(
    x=alt.X('count()', scale=alt.Scale(domain=(0,20))),
    y = alt.Y('group:O', scale=alt.Scale(domain=(groups))),
    color=color,
    opacity=alt.condition(selection2, alt.value(1), alt.value(0.25)),
).transform_filter(selection).add_selection(selection2)

# concatenate all layers into one multi-view layout
alt.vconcat(edges+nodes, bars)

alt.VConcatChart(...)

## Your Turn

  1. Add [eigenvector centrality](https://networkx.github.io/documentation/stable/reference/algorithms/generated/networkx.algorithms.centrality.eigenvector_centrality.html#networkx.algorithms.centrality.eigenvector_centrality) to the node atttributes. Print out the top ten characters based on this centraility measure. 
  2. Draw the graph using the `shell_layout` and the nodes having the same color and shape.  Mouseover will give the name. 
  3. Draw the graph using the `shell_layout` and add encoding `node_size` according to eigenvector centrality and color according to the group. Mouseover will give the name. 

In [25]:
nxa.draw_networkx(G, node_tooltip='name').properties(width=500, height=500)

alt.LayerChart(...)

In [26]:
# 1
eigen = nx.eigenvector_centrality(G)
nx.set_node_attributes(G, eigen, 'centrality')

sorted_eigen = sorted(eigen.items(), key=lambda x: x[1], reverse=True)

print("Top 10 nodes by eigen:")

for d in sorted_eigen[:10]:
  print( " - {}: {} ".format(G.nodes[d[0]]["name"], d[1]) )

Top 10 nodes by eigen:
 - Gavroche: 0.31783893977497674 
 - Valjean: 0.2676181759885393 
 - Enjolras: 0.26717863282356663 
 - Marius: 0.25911114534178753 
 - Bossuet: 0.24213078637474134 
 - Courfeyrac: 0.23246719717021405 
 - Bahorel: 0.22155360926119963 
 - Joly: 0.22155360926119963 
 - Combeferre: 0.21073457488115616 
 - Feuilly: 0.21073457488115616 


In [27]:
# 2
pos = nx.shell_layout(G)

chart = nxa.draw_networkx(
    G, pos=pos, node_tooltip='name'
).properties(width=500, height=500)

chart1 = nxa.draw_networkx( G, pos=pos,
    node_size= 200,
    node_color='group:N',
    width='value:Q',
    node_tooltip='name',
    linewidths=0
)

chart

alt.LayerChart(...)

In [28]:
# 3
chart = nxa.draw_networkx(
    G, pos=pos, node_tooltip='name', node_color='group:N', cmap="category10", node_size='centrality'
).properties(width=500, height=500)

chart

alt.LayerChart(...)

## Sources


Tutorials & Documentation
- [Tutorial — NetworkX 2.4 documentation](https://networkx.github.io/documentation/stable/tutorial.html)
- [Exploring and Analyzing Network Data with Python](https://programminghistorian.org/en/lessons/exploring-and-analyzing-network-data-with-python)
- https://github.com/Zsailer/nx_altair
